In [ ]:
from subtitle_process import fetch_subtitle
import getpass
import os
import json
import re
from templates import *
from functions import inference

from langchain.prompts import PromptTemplate

In [ ]:
# 设置临时环境变量
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("bilibili_ssid")

In [ ]:
with open("url.json", 'r') as file:
    data = json.load(file)

with open("subtitle.json", 'r') as f:
    subtitles = json.load(f)

if len(subtitles) == 0:
    subtitles = []
    for bv in data:
        subtitles.append(fetch_subtitle(bv))
# subtitles

In [ ]:
def extract_question_explanations(responses):
    questions_and_explanations = []
    for response in responses:
        pattern = re.compile(r"### 原题：\n(.+?)\n\n### 讲解：\n(.+?)\n\n### 相似题一：\n(.+?)\n\n### 相似题一讲解：\n(.+?)(?:\n\n|$)", re.DOTALL)
        matches = pattern.findall(response)

        for match in matches:
            questions_and_explanations.append({
                "original_question": match[0].strip(),
                "original_explanation": match[1].strip(),
                "similar_question_1": match[2].strip(),
                "similar_explanation_1": match[3].strip(),
            })

    with open('problem_set.json', 'w', encoding='utf-8') as f:
        json.dump(questions_and_explanations, f, ensure_ascii=False, indent=4)


async def subtitle2raw(subtitle):
    # 从subtitle里面获取题目，解析，相似题目，相似题目解析
    
    raw_output = await inference({"subtitle": subtitle}, template_sub_1_similar)
    return raw_output

    

In [ ]:
problem_set = []
for subtitle in subtitles:
    ps = await subtitle2raw(subtitle)
    problem_set.append(ps)

In [ ]:
extract_question_explanations(problem_set)